In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
#from nltk.probability import FreqDist
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
import numpy as np
from sklearn.decomposition import PCA
#import glob
from os import listdir
from os.path import isfile, join
import re
import itertools
import matplotlib.pyplot as plt
import math
from nltk import tokenize
#pd.options.display.float_format = '{:.2f}'.format

In [4]:
def Book_Chunker(books):
    m=""
    book_chunks=[]
    for i in range(len(books)):
        soup = BeautifulSoup(open(books[i]), 'html5lib')
        #soup = BeautifulSoup(open(book), 'html5lib')
        x1=soup.find_all('p')
        for j in x1:
            m=m+j.text #string containing the whole book
        dict = {}
        no_parts = 10 #no.of chunks per book
        length = math.floor(len(m)/no_parts)
        sentences = tokenize.sent_tokenize(m) #sentence tokenizing text to avoid breaking the sentences during chunking.

        flag = 0
        c = 0

        for k in range(no_parts):
          content = ""
          flag = 0
          while flag == 0:
            dict["part"+str(k)] = content #dictionary containing approximately equal chunks 
            content = content + sentences[c]
            c = c + 1
            if (len(content) > length) or (c >= len(sentences)):    
              flag = 1
              c = c-2
        book_chunks.append([ v for v in dict.values() ]) # list containing chunks of all books
    return book_chunks




In [5]:
   
def Book_Vectors(book_chunks): 
    book_vectors=[]
    for i in range(len(book_chunks)):
            chunk_vectors=[] 
            for j in range(len(book_chunks[i])):            
                result = [word_tokenize(t) for t in sent_tokenize(book_chunks[i][j])]
                results=[[w.lower() for w in x if w.isalpha()] for x in result] #sentence tokenized input for Word2vec without punctuation 
                model = Word2Vec(results,size=300,sg=1,window=5,iter=10) #Word2vec Skipgram model trained individually per chunk                
                v=[]

                words = list(model.wv.vocab) #vocabulary of the model
                for word in words:
                    #v.append(model[word])
                    v.append((model.syn1neg[model.wv.vocab[word].index]+model[word])/2) #average of context and center wordvectors for each word 
                chunk_vectors.append(sum(v)/len(v)) #average of all word vectors per chunk
            book_vectors.append(chunk_vectors) # list containing chunk vectors of all books
    return book_vectors,results


In [6]:
def Vectors_Transform(book_vectors):
    
    pca = PCA(n_components=2) #each chunk vector is reduced to 2 dimension
    final_vectors=[]
    for i in range(len(book_vectors)):
        X1 = pca.fit_transform(book_vectors[i]) # reducing the dimensions of all vectors using PCA
        final_vectors.append([item for sublist in X1 for item in sublist]) #list of all transformed vectors for all books

    df = pd.DataFrame()   
    df=df.append(pd.DataFrame(final_vectors)) #each row of df represents genre feature of a book
    #print(final_vectors)
    b=[(i.split('\\')[-1]).split('-')[0]  for i in books]
    df.insert(loc=0, column='Book_Id', value=b)
    #df.to_csv('D:\SEM-3\SIMFIC_Project\Features.csv',index=False) #write to CSV
    df.to_csv('D:\SEM-3\SIMFIC_Project\Toy3.csv', index=False,float_format='%11.6f')
    return df


In [12]:
#path=r'D:\SEM-3\SIMFIC_Project/pg22824-content.html' #path containing htmls files of books
paths=r'D:\New folder'
books = [join(paths, f) for f in listdir(paths) ]
a,results=Book_Vectors(Book_Chunker(books))

#path=r'D:\SEM-3\SIMFIC_Project\Books\pg141JaneAusMansPark-content.html'


C:\Users\Chaitanya Upadrasta\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `syn1neg` (Attribute will be removed in 4.0.0, use self.trainables.syn1neg instead).
  
C:\Users\Chaitanya Upadrasta\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [20]:
m=''
soup = BeautifulSoup(open(books[1]), 'html5lib')
#soup = BeautifulSoup(open(book), 'html5lib')
x1=soup.find_all('p')
for j in x1:
    m=m+j.text #string containing the whole book
sentences = tokenize.sent_tokenize(m) #sentence tokenizing text to avoid breaking the sentences

In [19]:
sentences

[<p>Die Brise wurde stÃ¤rker, und die Passagiere hatten bald alles Andere in dem einen GefÃ¼hl der Landung vergessen. Das niedere Ufer, an dem sich freilich noch immer keine Berge entdecken lieÃŸen, so viel auch die Leute mit bewaffneten und unbewaffneten Augen danach spÃ¤hten, trat dabei mehr und mehr heraus. Dort lieÃŸ sich schon die Einfahrt selbst unterscheiden, wo der gewaltige Mississippi in den Golf von Mexico mÃ¼ndet, und Â»sÃ¼ÃŸes WasserÂ« kam ihnen von da wieder aus dem Land ihrer Sehnsucht entgegen - ein FluÃŸ war es, der sie bald mit beiden Armen liebend umfangen sollte, und die See, die weite Ã¶de See lag hinter ihnen, wie ein schwerer Traum.
 </p>,
 <p>Selbst die CajÃ¼tenpassagiere gingen jetzt ernstlich daran ihre Sachen zu packen und sich auf eine baldige Landung vorzubereiten, und die Matrosen waren unter der Leitung des Untersteuermanns emsig damit beschÃ¤ftigt die beiden, auf der Back liegenden Anker Â»klarÂ« zu machen, und die groÃŸe mÃ¤chtige Kette gliederweis hera

In [10]:
df=Vectors_Transform(a)

In [12]:
df=pd.read_csv('D:\SEM-3\SIMFIC_Project\Toy_w=5,300.csv')

In [13]:
df

,Book_Id,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,pg105JaneAusPersuatn,-0.001407,0.002297,0.000092,0.000929,0.000086,-0.000773,0.002336,-0.000230,0.000336,...,0.000752,-0.000394,-0.000112,0.000305,0.000052,0.000118,-0.000145,0.000176,-0.001989,-0.002114
1,pg108DoyleReturnSherlk,0.004921,0.004470,0.001421,-0.000914,0.004277,-0.003565,0.001548,-0.001281,-0.000362,...,-0.001061,0.000021,-0.002873,0.000169,-0.002525,0.000603,-0.002838,0.000097,-0.002508,0.000474
2,pg1155Agatha2,0.007445,0.005414,0.003148,-0.005292,0.004563,-0.003601,0.001002,0.000550,-0.002529,...,-0.002666,-0.000208,-0.002215,0.000564,-0.003175,0.000708,-0.002612,0.000404,-0.002960,0.000169
3,pg1342JaneAustenPP,0.007202,0.008538,0.008359,-0.002968,-0.000762,-0.001681,-0.004493,0.001709,-0.005045,...,-0.004753,0.000755,-0.003851,0.000573,0.002213,-0.004143,0.000161,-0.001887,0.000969,-0.003247
4,pg1400DickensGreatExp,0.013567,0.001765,0.005253,-0.000260,-0.005339,0.002163,-0.006168,-0.001393,-0.005507,...,0.003614,-0.003125,0.005238,-0.001512,-0.004973,-0.008697,-0.002780,-0.004394,-0.002903,0.011228
5,pg141JaneAusMansPark,0.014584,-0.004273,0.006958,-0.002261,-0.004595,-0.006582,-0.004368,-0.007458,0.003632,...,0.004792,0.004775,-0.012472,0.002540,-0.006380,-0.001847,-0.003555,0.006536,0.001404,-0.001953
6,pg158JaneAustenEmma,0.019854,0.003815,-0.006192,0.007255,-0.006151,-0.000045,-0.003616,-0.008353,-0.003530,...,-0.002287,0.009457,-0.005936,0.011424,0.004330,-0.001096,0.008490,-0.004774,-0.004962,-0.007667
7,pg161SJaneAusSensSensi,0.017070,0.017000,-0.009858,0.001257,-0.010632,0.002700,0.006619,-0.008128,-0.011912,...,-0.007280,0.003590,0.000135,-0.002421,0.011013,-0.007511,0.002735,-0.002267,0.002110,-0.009284
8,pg19337DickensChristCarl,0.017832,-0.000613,-0.002383,-0.009021,-0.002991,-0.008817,-0.001276,0.008953,-0.004180,...,-0.012730,-0.006801,0.001976,-0.006207,0.014209,0.002880,-0.002105,0.008184,-0.008350,0.018152
9,pg2097DoyleSignFour,0.017936,-0.012271,-0.004735,-0.004697,-0.011414,-0.006846,0.006221,0.003731,-0.003567,...,-0.011685,-0.008567,0.014312,0.003865,0.006183,0.011012,-0.008725,0.015057,-0.004526,0.000204


In [49]:
df[df['Book_Id']=='pg108DoyleReturnSherlk'].index[0]

0

In [53]:
df.iloc[j]['Book_Id']

'pg834DoyleMemoirsSherlk'

In [55]:
from scipy.spatial import distance
from scipy.spatial.distance import cosine
from heapq import nsmallest
import pandas as pd
import numpy as np
i=df[df['Book_Id']=='pg108DoyleReturnSherlk'].index[0]
s=[]
for j in range(5):
    if(j!=i):
        print(df.iloc[j]['Book_Id'],cosine(df.iloc[i][1:20], df.iloc[j][1:20]))
#print('-------------------')

pg141JaneAusMansPark 0.7652006640272975
pg158JaneAustenEmma 1.0074607400407878
pg537DoyleTerrorTales 0.9811394997717419
pg834DoyleMemoirsSherlk 1.3667117285541361


In [18]:
from scipy.spatial import distance
from scipy.spatial.distance import cosine

query = df[df['Book_Id']=='pg2097DoyleSignFour'].index[0]

def test_similarity(query,frame):
  sim={}
  for item in df.index.to_list():
    sim[df.loc[item]['Book_Id']] = cosine(df.iloc[query][1:20],df.iloc[item][1:20])
    
    
    # print(1/(1+distance.euclidean(query,df.loc[item])))
  return sim

result = test_similarity(query,df)
final_result = sorted(result, key=result.get)
final_result

['pg2097DoyleSignFour',
 'pg2147EdgarPoe1',
 'pg19337DickensChristCarl',
 'pg4765Galsw3',
 'pg2148EdgarPoe2',
 'pg4397Galsw1',
 'pg1155Agatha2',
 'pg161SJaneAusSensSensi',
 'pg98DickensTaleCities',
 'pg2919Galsw5',
 'pg1400DickensGreatExp',
 'pg786DickensHardTimes',
 'pg537DoyleTerrorTales',
 'pg158JaneAustenEmma',
 'pg141JaneAusMansPark',
 'pg108DoyleReturnSherlk',
 'pg2852DoyleHound',
 'pg34970HermanPierre',
 'pg834DoyleMemoirsSherlk',
 'pg1342JaneAustenPP',
 'pg2911Galsw2',
 'pg4045HermanOmmo',
 'pg105JaneAusPersuatn',
 'pg21816HermanConfidence',
 'pg863Agatha1',
 'pg2149EdgarPoe3',
 'pg2150EdgarPoe4',
 'pg2701HermanMobyDick',
 'pg730DickensOliverTw',
 'pg766DickensDavidCopfld',
 'pg2684Galsw4']

In [82]:
result


{'pg108DoyleReturnSherlk': 0.0,
 'pg141JaneAusMansPark': 0.03246102635497365,
 'pg158JaneAustenEmma': 0.056653208860388306,
 'pg537DoyleTerrorTales': 0.053914905617763305,
 'pg834DoyleMemoirsSherlk': 0.0671776343118108}

In [7]:
from scipy.spatial import distance
from scipy.spatial.distance import cosine
from heapq import nsmallest
import pandas as pd
import numpy as np

def second_smallest(n):
    return nsmallest(2, n)[-1]
def third_smallest(n):
    return nsmallest(3, n)[-1]
def fourth_smallest(n):
    return nsmallest(4, n)[-1]
def fifth_smallest(n):
    return nsmallest(5, n)[-1]


tcount=0
fcount=0
#df=pd.read_csv('D:\SEM-3\SIMFIC_Project\Features1.csv')
for i in range(31): #looping over books
    s=[]
    for j in range(31):
        #print(distance.euclidean(df.iloc[i][:19], df.iloc[j][:19]))
        if(cosine(df.iloc[i][1:10], df.iloc[j][1:10])==0):
            a=100000
        else:
            a=cosine(df.iloc[i][1:10], df.iloc[j][1:10]) # cosine similarity of book vectors
        s.append(a)
    #print(np.argmin(s))
    tstr=df.iloc[i]['Book_Id']
    #print(tstr)


            
    if 'Doyle' in tstr:
        if ('Doyle' in df.iloc[np.argmin(s)]['Book_Id']) or ('Doyle' in df.iloc[s.index(second_smallest(s))]['Book_Id'])or ('Doyle' in df.iloc[s.index(third_smallest(s))]['Book_Id']) or ('Doyle' in df.iloc[s.index(fourth_smallest(s))]['Book_Id'])or ('Doyle' in df.iloc[s.index(fifth_smallest(s))]['Book_Id']):
                #print ("true",tstr,'Doyle')
                tcount+=1

        else:
            #print('false',tstr,'Doyle')
            fcount+=1

 
    elif 'Jane' in tstr:
        if ('Jane' in df.iloc[np.argmin(s)]['Book_Id']) or ('Jane' in df.iloc[s.index(second_smallest(s))]['Book_Id'])or ('Jane' in df.iloc[s.index(third_smallest(s))]['Book_Id'])or ('Jane' in df.iloc[s.index(fourth_smallest(s))]['Book_Id'])or ('Jane' in df.iloc[s.index(fifth_smallest(s))]['Book_Id']):
                #print ("true",tstr,'Jane')
                tcount+=1
        else:
            #print('false',tstr,'Jane')
            fcount+=1
        
    elif 'Herman' in tstr:
        if ('Herman' in df.iloc[np.argmin(s)]['Book_Id']) or ('Herman' in df.iloc[s.index(second_smallest(s))]['Book_Id'])or ('Herman' in df.iloc[s.index(third_smallest(s))]['Book_Id'])or ('Herman' in df.iloc[s.index(fourth_smallest(s))]['Book_Id'])or ('Herman' in df.iloc[s.index(fifth_smallest(s))]['Book_Id']):
                #print ("true",tstr,'Herman')
                tcount+=1
        else:
            #print('false',tstr,'Herman')
            fcount+=1
    elif 'Galsw' in tstr:
        if ('Galsw' in df.iloc[np.argmin(s)]['Book_Id']) or ('Galsw' in df.iloc[s.index(second_smallest(s))]['Book_Id'])or ('Galsw' in df.iloc[s.index(third_smallest(s))]['Book_Id'])or ('Galsw' in df.iloc[s.index(fourth_smallest(s))]['Book_Id'])or ('Galsw' in df.iloc[s.index(fifth_smallest(s))]['Book_Id']):
                #print ("true",tstr,'Galsw')
                tcount+=1  
        else:
           # print('false',tstr,'Galsw')
            fcount+=1
    elif 'Dickens' in tstr:
        if ('Dickens' in df.iloc[np.argmin(s)]['Book_Id']) or ('Dickens' in df.iloc[s.index(second_smallest(s))]['Book_Id'])or ('Dickens' in df.iloc[s.index(third_smallest(s))]['Book_Id'])or ('Dickens' in df.iloc[s.index(fourth_smallest(s))]['Book_Id'])or ('Dickens' in df.iloc[s.index(fifth_smallest(s))]['Book_Id']):
                #print ("true",tstr,'Dickens')
                tcount+=1   
        else:
            #print('false',tstr,'Dickens')
            fcount+=1
    elif 'Edgar' in tstr:
        if ('Edgar' in df.iloc[np.argmin(s)]['Book_Id']) or ('Edgar' in df.iloc[s.index(second_smallest(s))]['Book_Id'])or ('Edgar' in df.iloc[s.index(third_smallest(s))]['Book_Id'])or ('Edgar' in df.iloc[s.index(fourth_smallest(s))]['Book_Id'])or ('Edgar' in df.iloc[s.index(fifth_smallest(s))]['Book_Id']):
                #print ("true",tstr,'Edgar')
                tcount+=1     
        else:
            #print('false',tstr,'Edgar')
            fcount+=1
    elif 'Agatha' in tstr:
        if ('Agatha' in df.iloc[np.argmin(s)]['Book_Id']) or ('Agatha' in df.iloc[s.index(second_smallest(s))]['Book_Id'])or ('Agatha' in df.iloc[s.index(third_smallest(s))]['Book_Id'])or ('Agatha' in df.iloc[s.index(fourth_smallest(s))]['Book_Id']) or ('Agatha' in df.iloc[s.index(fifth_smallest(s))]['Book_Id']):
                #print ("true",tstr,'Agatha')
                tcount+=1     
        else:
            #print('false',tstr,'Agatha')
            fcount+=1
